# Deep Learning in Keras
This will be a tutorial on how to use [Keras](http://keras.io/) for deep learning. I will be following the example code given [here](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) but will make sure to document more as I am to target this tutorial for people new to deep learning or who simply want to learn a bit more about it.

## Installation
Keras is relatively simple to install. Simply follow the instructions [here](http://keras.io/#installation). Note that Numpy and Scipy come preinstalled with iPython so you should not need to install those again. Should you encounter any problems Google is your friend!

## Imports
Keras is relatively lightweight unless you want to use a lot of different layer types. We will get into the layers being used and others available in Keras a bit later in the tutorial.

In [1]:
import numpy as np

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using Theano backend.


Using gpu device 0: GeForce GTX 980 (CNMeM is disabled)


# Reproducability
It is always good to seed your network before doing anything. Initialization of the network weights is random(usually) and it is good practice to seed so your results are reproducible and not subject to fluctuations in weight initialization. 

In [2]:
np.random.seed(12345)  # for reproducibility

## The Dataset
We will be doing a classic problem that deep learnign excels at which is learning to classify pictures of numbers. The dataset we will be working on is [MNIST](http://yann.lecun.com/exdb/mnist/). We will have a set of 10 different digits to be classified as 0-9. Keras comes with the MNIST dataset and parsing functionality built in so we will exploit this fully! This is about deep learning not data munging :).

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

('X_train shape:', (60000, 1, 28, 28))
(60000, 'train samples')
(10000, 'test samples')


## Hyperparameter Settings
Neural networks have quite a few hyperparameters that need to be set. This is good in that it leaves a lot of room for tweaking your model but unfortunate that even changing 1 parameters slightly can drastically reduce performance. Tuning these parameters is an art and I reccomend you take a look at [this](https://www.coursera.org/course/neuralnets) course to learn about all of the different parameters and smart ways to set them. Nonetheless the default parameters below should work just fine and I will go into more depth as to what they mean:

- **batch_size** - Mini-batch learning is a really popular way to train networks when the dataset is redundant and plentiful. We exploit the fact that we have redundancy and a lot of data by sampling the dataset every for every update to the network. This is extremely efficient as we do not need to go through every training case per network update which would take an eternity. This parameter will set how large the batch size should be, in this case 128 images per network update. 
- **num_classes** - More intuitively this is just the number of distinct labels in your data. Since we are classifying digits we will have 10 different labels.
- **num_epochs** - This will specify now many times we send the training data through the network to train. Be careful, you might be tempted to set this parameter high which will almost always increase training set accuracy but this will make the network overfit and reduce generizability to other data. 

In [4]:
batch_size = 128
num_classes = 10
num_epoch = 2

In [5]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

## Making a Network Architecture
This is where the meat of the network is created. Keras makes the process of creating a network easy by being able to simply stack layers on top of other layers and specify activation functions as neccesary. 

But how do we know which types of layers to use, how big to make them, what types of activation functions to use, etc... This is a hard question with a very simple answer and a not so simple answer. The simple answer is to simply copy someone elses network architecture which has proven to work. The not so simple answer is to consider the problem you are trying to solve and consider what the different layers would do mathematically.

**Example:** For image recognition convolutional neural networks excel. These architectures usually follow a common pattern which looks a lot like this:
![pic](http://eblearn.sourceforge.net/lib/exe/lenet5.png)

The key thing to notice is that we have very large kernels followed by fewer and fewer as we go. The first couple of layers have been shown to extract simple features acting similar to different types of edge detectors while each following layer can extract more and more complex features. This type of architecture is biologically inspired as this is how our brain does image processing! [ needs reference ] Using successively smaller layers forces the network to learn to integrate the previous layers and creates a more complex model. 

There are entire courses, books, and papers on how a network should be architectured and tweaked so I leave it to you to look for those and try and find an architecture that suits your needs! For anything image related I reccomend following the template above or similar architectures such as [GoogLeNet](http://arxiv.org/pdf/1409.4842.pdf).

## Our Architecture
We will be using a much simpler architecture as our problem is easy enough. The layers we will use will be convolutional, pooling, and dense. The activation functions we will use will be reLu for the inner layers and a softmax output classifier. Our error function will be cross-entropy. I will give a brief description of what these layers will do:
- **convolutional** - If you are unfamiliar with what a convolution is [here](http://www.songho.ca/dsp/convolution/convolution2d_example.html) is a good example but I do reccomend going deeper and understanding the math. At a high level you can consider this layer as filtering the input image to extract features(corners, edges, faces, dogs, etc...). 
- **dense** - This is a fully connected layer of single neurons. This is what I'd assume is classically thought of when someone mentions neural networks. You have all neurons to the previous layer connected to all neurons on the next layer. An interesting curiosity to think about is if we could replicate what a convolutional layer is doing with a LOT of single neurons. Think about it and if you are curious I reccomend Googling around, I think its a good thought exercise :)
- **pooling** - Pooling is simply the operation of downsampling the inputs from the previous layer before feeding them into the next layer. This can be done in many ways and each one has it's upside. Common ways are using max or mean operations of a grid of the image to reduce the image size. This has 2 consequences; it reduces the input size to the next layer which reduces computation and it introduces a sort of translation invariance in the image. If you think about it there are 8 ways to shift a pixel in an image, if you have a pooling filter of size 2x2 you are going to get 3/8 that return the same answer instead of only 1/8. This gets better as you making the pooling filter larger.

**reLu activation** - It is common for neural nets to be taught with sigmoid activation functions which is fine and biologically based. But it has been [shown](http://www.cs.toronto.edu/~fritz/absps/reluICML.pdf) that using rectified linear units helps with the [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem) in very deep networks. I would almost always use rectified linear units but again, experiment and use what has been proven to work!

**softmax output**- Normally an output unit will output a probability of the input belonging to its class. This can give problems as the outputs of all of the output neurons is not a probability distribution. The softmax converts these output probabilities to a probability distribution so one can simply take the most probable class as the final output. 

**Dropout** - Dropout is the act of excluding a percentage of neurons in the update process. So if we have 28 neurons in one layer we could use only 14 of them in the next layer. This has been [shown](http://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) to improve generalizability of the network and is almost always reccomended if your dataset is large enough. A simple way to think about it is that if only half the neurons are used per training case then those neurons must learn relavant features that are unique from the other neurons and general enough to be used in multiple cases. 

In [6]:
#Defines a sequential model where we stack layers to construct our network
#One can also define a graph to construct the network. Take a look in the
#Keras documentation here: http://keras.io/models/
model = Sequential()

#You can check all layer input parameters here: http://keras.io/layers/core/

#Convolutional layer using 32 filters of size 3x3 using relu activation
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(1,28,28)))
model.add(Activation('relu'))

#Convolutional layer using 32 filters of size 3x3 using relu activation
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))

#MaxPooling layer with pooling filter size of 2x2
model.add(MaxPooling2D(pool_size=(2, 2)))

#Dropout 25% of the neurons
model.add(Dropout(0.25))

#Flatten output to a 1D vector and feed into dense layer with relu activation
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

#Dropout 50% of the neurons
model.add(Dropout(0.5))

#Final layer before classification. Dense connected to 10 neurons which
#with softmax will give us a probability distribution of the digits this
#image could be
model.add(Dense(num_classes))
model.add(Activation('softmax'))

## Training
The process of training the network is pretty simple. We feed in batches of images if using mini-batch and the network will pop out and output. We measure the error using whatever error metric we wish(in this case cross-entropy) and then backpropogate this error through the network to change the weights connecting the layers. After feeding the batches of images through the network the number of epochs we specified we will have a trained neural net!

![black box](http://www.kdnuggets.com/wp-content/uploads/blackbox.jpeg)

This probably feels a little black-boxey and it should! Deep learning is a really *deep* field and there is a lot going on behind the scenes. Here is my attempt to open the box slightly.

The training process is reducing the error rate of classification by measuring its current error via the metric specified. Easy and understandable. 

### Backpropogation
The complex part people take little time to understand is the process of backpropogating this error through the network. To understand the backpropogation algorithm I reccomend going through the first few modules of this [course](https://www.coursera.org/course/neuralnets) and [this]() one as well. At a VERY high level you can consider the backpropogation an application of the chain rule in calculus. We have a error and wish to calculate the error that a particular neuron contributed which is the overall error with respect to this neurons weight vector. This is a derivative, and relatively simple to calculate. Once we have this error we then update the neurons weight to the next layers using an activation function that we specifiy. Different activation units have different mathematical properties so I will exclude the mathematics behind it but I encourage you to do your own digging if you are interested. 

That's it. We update the weight and go to the previous layer and do the same thing until we reach the input layer, which we then feed in the next batch of images and do it all over. Easy peezy!

Now I HIGHLY reccomend you go and make a simple ANN from scratch. Implementing a sigmoid activation functions, dense layers, and backprop with squared error should be a good place to start and there are THOUSANDS of resources to help you. Once you have done that you should have a better grasp of backprop and neural nets in general! But for those who simply want to use them we will continue with training our digit classifier!

In [7]:
#The parameters here are the error function and optimizer function. Documentation
#on both of these can be found http://keras.io/optimizers/ and http://keras.io/objectives/
model.compile(loss='categorical_crossentropy', optimizer='adadelta')

#Training the model!
#This may take a while... :)
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=num_epoch, 
          show_accuracy=True, verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 36s - loss: 0.2350 - acc: 0.9273 - val_loss: 0.0607 - val_acc: 0.9800
Epoch 2/2
60000/60000 [==============================] - 34s - loss: 0.0894 - acc: 0.9733 - val_loss: 0.0403 - val_acc: 0.9863


In [8]:
score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('Test score:', 0.040320074847433718)
('Test accuracy:', 0.98629999999999995)


In [ ]:
#98% accuracy! Note bad for your first network :) . You can get this to be closer 
#to 100% with more epocs but at the sacrifice of time. 

## Exporting Weights
Want to use this network somewhere else but don't want to train all over again? Want to initialize another network smartly before training on another dataset? Luckily Keras gives you the power to export the learned weights of a network so you can import them again in other code. 

In [9]:
model.save_weights('keras_intro_weights')

#You can now use model.load_weights("keras_intro_weights") to load the learned weights
#into a model that has the same architecture as the one created above. 

# Interesting Points
This is a place to note subtle points that I did not touch on throughout the tutorial but are important in their own right. If you feel something else should be added then by all means go ahead! Please avoid touching the code unless it is broken though.

- **Dividing the image pixels by 255**. Notice that we are normalizing the image between 0-1 and we are using rectified linear units. This is on purpose! If we had a large input space (0-255) then the gradients in backprop would tend to explode as we are using a linear function. Normalizing can help tremendously in all machine learning tasks.

## Going Further
Here is what you should do next!
- **Determine if deep learning is right for your problem** It is tempting to say it is always the right solution to the problem but avoid this. If your input space is small ( <1000 variables id say) then consider using normal linear or logistic regression.
- **Look at other types of networks** Some interesting ones are Recurrent Neural Networks(RNN), Restricted Boltzman Machines, Echo Networks, and Long-Short Term Memory(LSTM).
- **Implement backpropogation at LEAST once** Seriously, do it. You will be doing yourself a big favor in the long run, especially if you want to start looking into research papers. 
- **Check out the Keras documentation** There are a lot of things already built in to Keras so don't try and reinvent the wheel. 
- **Check out other neural network frameworks** There are a lot! Each one is different and suits a specific need. Determine which one suits your needs best. There will be links to popular ones in the tutorial repository.